# GIF Processing Notebook

This notebook provides functionality to:
1. Resize GIFs to specified dimensions
2. Add text overlay at selected positions
3. Add image overlay at selected positions
4. Compress the resulting GIF

In [ ]:
# Import required libraries
from PIL import Image, ImageDraw, ImageFont
import os
from IPython.display import display, Image as IPImage
import numpy as np

In [ ]:
import matplotlib.font_manager as fm
class FontManager:
    """Manages fonts for the GIF processor"""

    def __init__(self):
        self.fonts = {
            'default': None,  # Will be initialized with Ubuntu font
            'matplotlib': {}  # Will be populated with matplotlib fonts
        }
        self._initialize_fonts()

    def _initialize_fonts(self):
        # Load built-in Ubuntu font
        # Load matplotlib fonts
        font_names = ['serif', 'sans-serif', 'monospace']
        for name in font_names:
            try:
                font_path = fm.findfont(fm.FontProperties(family=name))
                self.fonts['matplotlib'][name] = font_path
            except:
                continue

    def get_font(self, font_name='default', size=30):
        """Get a font by name and size"""
        try:
            if font_name == 'default':
                return ImageFont.truetype(self.fonts['default'], size)
            elif font_name in self.fonts['matplotlib']:
                return ImageFont.truetype(self.fonts['matplotlib'][font_name], size)
            else:
                raise ValueError(f"Font '{font_name}' not found")
        except Exception as e:
            print(f"Error loading font {font_name}: {str(e)}")
            return ImageFont.load_default()

    def list_fonts(self):
        """List all available fonts"""
        available_fonts = ['default'] + list(self.fonts['matplotlib'].keys())
        return available_fonts

In [ ]:
from io import BytesIO


class GIFProcessor:
    def __init__(self, gif_path):
        """Initialize with path to GIF file"""
        self.gif = Image.open(gif_path)
        self.frames = []
        self.durations = []

        # Extract all frames and their durations
        try:
            while True:
                self.frames.append(self.gif.copy())
                self.durations.append(self.gif.info.get('duration', 100))
                self.gif.seek(self.gif.tell() + 1)
        except EOFError:
            pass

    def resize(self, width, height):
        """Resize all frames to specified dimensions"""
        self.frames = [frame.resize((width, height), Image.Resampling.LANCZOS)
                      for frame in self.frames]
        return self

    def make_square(self, size=None):
        """
        Crop the GIF to a square from the center.
        If size is provided, the output will be resized to size x size.
        If size is None, the square will be sized to the smaller dimension.
        """
        if not self.frames:
            return self

        # Get dimensions from first frame
        width, height = self.frames[0].size

        # Calculate crop box for square
        if width > height:
            # Landscape orientation
            left = (width - height) // 2
            top = 0
            right = left + height
            bottom = height
        else:
            # Portrait orientation
            left = 0
            top = (height - width) // 2
            right = width
            bottom = top + width

        # Apply crop to all frames
        self.frames = [frame.crop((left, top, right, bottom)) for frame in self.frames]

        # Resize if size is specified
        if size is not None:
            self.frames = [frame.resize((size, size), Image.Resampling.LANCZOS)
                         for frame in self.frames]

        return self

    def add_text(self, text, position, font_path=None, font_size=30,
                 color=(255, 255, 255), stroke_width=2, stroke_fill=(0, 0, 0)):
        """Add text overlay to all frames"""
        font_manager = FontManager()
        font = font_manager.get_font(font_name=font_path,size=font_size)

        for i, frame in enumerate(self.frames):
            # Convert to RGBA before drawing
            frame_rgba = frame.convert('RGBA')
            draw = ImageDraw.Draw(frame_rgba)
            draw.text(position, text, font=font, fill=color,
                     stroke_width=stroke_width, stroke_fill=stroke_fill)
            self.frames[i] = frame_rgba
        return self

    def add_image_overlay(self, overlay_path, position):
        """Add image overlay to all frames"""
        overlay = Image.open(overlay_path).convert('RGBA')

        for i, frame in enumerate(self.frames):
            frame_rgba = frame.convert('RGBA')
            frame_rgba.paste(overlay, position, overlay)
            self.frames[i] = frame_rgba
        return self

    def save(self, output_path, optimize=False, quality=100):
        """
        Save the processed GIF with optimization options

        Args:
            output_path (str): Path to save the GIF
            optimize (bool): Whether to optimize the GIF
            quality (int): Quality from 1 (worst) to 100 (best).
                         Lower quality means smaller file size.
        """
        if not self.frames:
            return

        # Convert frames to RGB mode for saving
        rgb_frames = []
        for frame in self.frames:
            rgb_frame = frame.convert('RGB')

            if optimize:
                # Calculate number of colors based on quality
                n_colors = max(min(256, int(256 * (quality / 100))), 2)

                # Convert to P mode (palette) with optimized palette
                rgb_frame = rgb_frame.quantize(
                    colors=n_colors,
                    method=Image.Quantize.MEDIANCUT,
                    dither=Image.Dither.FLOYDSTEINBERG
                )

            rgb_frames.append(rgb_frame)

        # Save with optimization
        rgb_frames[0].save(
            output_path,
            save_all=True,
            append_images=rgb_frames[1:],
            optimize=optimize,
            duration=self.durations,
            loop=0,
            format='GIF',
            # Additional optimization parameters
            disposal=2,  # Clear the frame before rendering the next
            quality=quality
        )
        print("Size on disk:", f"{os.path.getsize(output_path) / 1024 / 1024:.2f} MB")

    def display(self):
        """Display the current state of the GIF in the notebook"""
        temp_path = '_temp_display.gif'
        self.save(temp_path)
        display(IPImage(filename=temp_path))
        os.remove(temp_path)

    def get_size(self, optimize=False, quality=100):
        """Get the size of the processed GIF in bytes without saving to disk"""
        if self.frames:
            # Convert frames back to RGB mode for saving
            rgb_frames = [frame.convert('RGB') for frame in self.frames]
            with BytesIO() as buffer:
                rgb_frames[0].save(
                    buffer,
                    save_all=True,
                    append_images=rgb_frames[1:],
                    optimize=optimize,
                    quality=quality,
                    duration=self.durations,
                    loop=0,
                    format='GIF'
                )
                return buffer.tell()

## Usage Example

In [ ]:
# Example usage
GIF_PATH='/Users/mat3ra/Downloads/wave-visualization (7).gif'
NEW_SIZE_WIDTH=600
NEW_SIZE_HEIGHT=600

LOGO_PATH= "/Users/mat3ra/Library/CloudStorage/GoogleDrive-vsevolod.biryukov@exabyte.io/Other computers/My MacBook Pro/marketing/Transparent white_color.png"
LOGO_POSITION = (15,10)

FONT_SIZE=24
TEXT_1="Material ABC"
TEXT_1_POSITION_X=10
TEXT_1_POSITION_Y=NEW_SIZE_HEIGHT-20 - FONT_SIZE

FINAL_NAME="output.gif"

gif_processor = GIFProcessor(GIF_PATH)

# # Display original
# print("Original GIF:")
# gif_processor.display()

# Resize to 300x200
# gif_processor.resize(*NEW_SIZE)
gif_processor.make_square(size=NEW_SIZE_WIDTH)

font_manager = FontManager()

# List available fonts
print("Available fonts:")
print(font_manager.list_fonts())

font = "serif"
# Add text overlay
gif_processor.add_text(
    TEXT_1,
    position=(TEXT_1_POSITION_X, TEXT_1_POSITION_Y),
    font_path=font,
    font_size=FONT_SIZE,
    color=(255, 255, 255),
    stroke_width=2
)

# Add image overlay (e.g., a logo)
gif_processor.add_image_overlay(
    LOGO_PATH,
    position=LOGO_POSITION
)

QUALITY=100
# Display modified GIF
print("\nModified GIF:")
print(f"Size: {gif_processor.get_size(quality=QUALITY) / 1024 / 1024:.2f} MB")
gif_processor.display()

# Save with compression
gif_processor.save(
    FINAL_NAME,
    optimize=True,
    quality=QUALITY
)

